<center>
    <h1><b>Phân Tích và Dự Đoán Trạng Thái Tâm Lý Qua Miêu Tả Dạng Văn Bản</b></h1>
    <h2><b>Học phần: Học máy</b></h2>
    <h3><b>Giảng viên: TS. Cao Văn Chung</b></h3>
</center>

---
**Sinh viên thực hiện:**
- Trần Anh Minh - 21000242
- Dương Đức Vương - 22000133
- Nguyễn Tuấn Kiệt - 22000100

## Tiền xử lý dữ liệu

In [1]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as mp
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re as r
from nltk.stem import PorterStemmer

* Tải dữ liệu và quan sát trực quan

In [4]:
df=pd.read_csv("New Data.csv")

In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df.dropna(inplace=True)

In [7]:
max_count=df['status'].value_counts().max()

In [8]:
def resample_data(df):
    max_count = df['status'].value_counts().max()

    df_resampled = pd.DataFrame()

    for status in df['status'].unique():
        df_class = df[df['status'] == status]
        if len(df_class) < max_count:
            df_class_resampled = resample(df_class,
                                          replace=True,
                                          n_samples=max_count,
                                          random_state=42)
            df_resampled = pd.concat([df_resampled, df_class_resampled])
        else:
            df_resampled = pd.concat([df_resampled, df_class])

    return df_resampled


df = resample_data(df)

In [9]:
pt=PorterStemmer()
def preprocessing(x):
    l=[]

    text = r.sub(r'[^a-zA-Z0-9\s]', '', x.lower())
    for i in text.split():
        l.append(pt.stem(i.lower()))


    return " ".join(l)

In [10]:
df['statement']=df['statement'].apply(preprocessing)

In [11]:
x=df['statement']
y=df['status']

In [12]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,random_state=42,test_size=0.2)
x_train2,x_test2,y_train2,y_test2=train_test_split(x,y,random_state=42,test_size=0.3)
x_train3,x_test3,y_train3,y_test3=train_test_split(x,y,random_state=42,test_size=0.4)


## MULTI LAYER PERCEPTRON

* Vector hóa tập các tập train/ test bằng TF-TDF

In [13]:
# TF-IDF từng tập train, test
vec1 = TfidfVectorizer()
x_train_tfidf1 = vec1.fit_transform(x_train1)
x_test_tfidf1 = vec1.transform(x_test1)

vec2 = TfidfVectorizer()
x_train_tfidf2 = vec2.fit_transform(x_train2)
x_test_tfidf2 = vec2.transform(x_test2)

vec3 = TfidfVectorizer()
x_train_tfidf3 = vec3.fit_transform(x_train3)
x_test_tfidf3 = vec3.transform(x_test3)



* Số chiều từng tập train, test

In [14]:
print("Tập train 1 (4:1):")
print("Số chiều tập train:", x_train_tfidf1.shape)
print("Số chiều tập test:", x_test_tfidf1.shape)

print("Tập train 2 (7:3):")
print("Số chiều tập train:", x_train_tfidf2.shape)
print("Số chiều tập test:", x_test_tfidf2.shape)

print("Tập train 3 (6:4):")
print("Số chiều tập train:", x_train_tfidf3.shape)
print("Số chiều tập test:", x_test_tfidf3.shape)

Tập train 1 (4:1):
Số chiều tập train: (125068, 54552)
Số chiều tập test: (31268, 54552)
Tập train 2 (7:3):
Số chiều tập train: (109435, 51231)
Số chiều tập test: (46901, 51231)
Tập train 3 (6:4):
Số chiều tập train: (93801, 47660)
Số chiều tập test: (62535, 47660)


* Dùng thư viện để huấn luyện mô hình, phân loại tập test và đánh giá mô hình

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def train_evaluate_MLP(x_train, x_test, y_train, y_test, description=""):
    # Khởi tạo mô hình MLP với các siêu tham số cơ bản
    model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=300, random_state=42)
    
    # Huấn luyện mô hình
    model.fit(x_train, y_train)
    
    # Dự đoán trên tập test
    y_pred = model.predict(x_test)
    
    # In kết quả đánh giá
    print(f"===== Đánh giá mô hình cho {description} =====")
    print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred))
    print("Báo cáo phân loại (Classification Report):")
    print(classification_report(y_test, y_pred))
    print("Ma trận nhầm lẫn (Confusion Matrix):")
    print(confusion_matrix(y_test, y_pred))
    print("\n")

# Đánh giá cho 3 tỷ lệ train/test
train_evaluate_MLP(x_train_tfidf1, x_test_tfidf1, y_train1, y_test1, description="Tập 1 (train:test = 4:1)")
train_evaluate_MLP(x_train_tfidf2, x_test_tfidf2, y_train2, y_test2, description="Tập 2 (train:test = 7:3)")
train_evaluate_MLP(x_train_tfidf3, x_test_tfidf3, y_train3, y_test3, description="Tập 3 (train:test = 6:4)")
